In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_label = 'labeling_2023'
collection_name = 'full_labeling_2023'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

#### Mengambil data dari MongoDB ke dalam dataframe

In [3]:
# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]  # Ganti dengan nama database Anda
labeling_2018_collection = db[collection_label]
full_labeling_2018_collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_2018_collection.find())
unlabeled_data = list(full_labeling_2018_collection.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeling_2018_collection.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


In [4]:
df_positif = df_unlabeled[df_unlabeled['predicted_label'] == 'positif']
df_negatif = df_unlabeled[df_unlabeled['predicted_label'] == 'negatif']
df_netral = df_unlabeled[df_unlabeled['predicted_label'] == 'netral']

In [5]:
df_positif

,_id,textOriginal,predicted_label
1,6651a6a8c14ed2b8f4863c65,wah hebat keren banget,positif
4,6651a6a8c14ed2b8f4863c68,rewind henti indonesia,positif
5,6651a6a8c14ed2b8f4863c69,keren konsep pakai kasus david kotak pandora k...,positif
7,6651a6a8c14ed2b8f4863c6b,merinding sih nontonya,positif
12,6651a6a8c14ed2b8f4863c70,keren moga gabung taun taun ikut,positif
...,...,...,...
9994,6651a6a8c14ed2b8f486636e,hebat,positif
9996,6651a6a8c14ed2b8f4866370,mantap keren,positif
9997,6651a6a8c14ed2b8f4866371,keren,positif
9998,6651a6a8c14ed2b8f4866372,moga video tragedi gunung marapi sumatra barat...,positif


In [6]:
df_negatif

,_id,textOriginal,predicted_label


In [7]:
df_netral

,_id,textOriginal,predicted_label
0,6651a6a8c14ed2b8f4863c64,kurang nih kreator gamenya ga lengkap,netral
2,6651a6a8c14ed2b8f4863c66,akting briyan furant sih,netral
3,6651a6a8c14ed2b8f4863c67,akhir liat miawaug rewind bener sih iklan hoto...,netral
6,6651a6a8c14ed2b8f4863c6a,nilai gua,netral
8,6651a6a8c14ed2b8f4863c6c,yo merinding asa flashback ke youtube rewind d...,netral
...,...,...,...
9980,6651a6a8c14ed2b8f4866360,tiap taun ntn youtube rewind kaca kaca trs mat...,netral
9983,6651a6a8c14ed2b8f4866363,ria sw dong,netral
9987,6651a6a8c14ed2b8f4866367,rewind circle,netral
9992,6651a6a8c14ed2b8f486636c,bryan furan,netral


#### Accuracy Test (soon)

In [8]:
# Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
if 'label' in df_unlabeled.columns:
    print("\nEvaluasi kinerja model:")
    print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
    print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
else:
    print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")

Data tidak berlabel tidak memiliki label asli untuk evaluasi.
